Week 6 : Real-World Applications: TF-IDF

The grader forces us to proceed in two steps, as it forbids using a unique reducer.
It would had been perfectly fine, as we emitted very few lines during mapping, but well...

So, first step : compute Dt

In [1]:
%%writefile mapper_dt.py


import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

# the word we search
searched_word = 'labor'

docs_with_word = 0

# let's read articles... One article per line, starting with id
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        print e
        continue

    # split words
    words_with_sws = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    # to lower case
    words_with_sws = [x.lower() for x in words_with_sws]

    if searched_word in words_with_sws:
        print "0\t1"

Overwriting mapper_dt.py


In [2]:
%%writefile reducer_dt.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer_dt.py


In [3]:
%%bash

OUT_DIR_DT=dt_dir

hdfs dfs -rm -r -skipTrash ${OUT_DIR_DT} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="TF IDF" \
    -files mapper_dt.py,reducer_dt.py \
    -mapper "python mapper_dt.py" \
    -reducer "python reducer_dt.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_DT} > /dev/null
    
# output result to console
hdfs dfs -cat ${OUT_DIR_DT}/part-* | grep -P '^0\t' | cut -f 2 > Dt.txt

18/11/07 16:59:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 16:59:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 16:59:11 INFO mapred.FileInputFormat: Total input files to process : 1
18/11/07 16:59:11 INFO mapreduce.JobSubmitter: number of splits:2
18/11/07 16:59:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1541602145497_0029
18/11/07 16:59:11 INFO impl.YarnClientImpl: Submitted application application_1541602145497_0029
18/11/07 16:59:11 INFO mapreduce.Job: The url to track the job: http://5c62422b412d:8088/proxy/application_1541602145497_0029/
18/11/07 16:59:11 INFO mapreduce.Job: Running job: job_1541602145497_0029
18/11/07 16:59:16 INFO mapreduce.Job: Job job_1541602145497_0029 running in uber mode : false
18/11/07 16:59:16 INFO mapreduce.Job:  map 0% reduce 0%
18/11/07 16:59:26 INFO mapreduce.Job:  map 100% reduce 0%
18/11/07 16:59:30 INFO mapreduce.Job:  map 100% reduce 100%
18/11/07 16:59:31 I

In [ ]:
Next step : compute TF*IDF

In [4]:
%%writefile mapper.py


import sys
import re
import math


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

# stop words
swf = open('stop_words_en.txt', 'r')
sws = swf.read().split('\n')
swf.close()

dtf = open('Dt.txt')
Dt = dtf.read()
Dt = float(Dt)

idf = float(1)/math.log(1+Dt)

# the word we search
searched_word = 'labor'
# the article for which we compute TF IDF
lookedup_article = 12

docs_with_word = 0

# let's read articles... One article per line, starting with id
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        print e
        continue

    # split words
    words_with_sws = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    # to lower case
    words_with_sws = [x.lower() for x in words_with_sws]
    # without stop words
    words = [x for x in words_with_sws if x not in sws]
    searched_word_count = words.count(searched_word)
    word_count = len(words)

    # we look for the tf idf for article 12
    # so, lets ouput these values only for this article...
    if article_id == lookedup_article:
        tf_idf = (float(searched_word_count)/float(word_count))*idf
        print "%s\t%s\t%s\t%g" % (article_id, searched_word_count, word_count,tf_idf)


Overwriting mapper.py


Running it !
We distribute the (small) stop words file

In [5]:
%%bash

OUT_DIR="coursera_mr_task"$(date +"%s%6N")
NUM_MAPPERS=4

# Stub code for your job

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="TF IDF" \
    -D mapreduce.job.reduces=0 \
    -D mapred.job.maps=${NUM_MAPPERS} \
    -files mapper.py,/datasets/stop_words_en.txt,Dt.txt \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
# output result to console
hdfs dfs -cat ${OUT_DIR}/part-* | grep -P '^12\t' | cut -f 4

# cleanup result dir
#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null



0.000350469


18/11/07 16:59:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 16:59:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 16:59:54 INFO mapred.FileInputFormat: Total input files to process : 1
18/11/07 16:59:55 INFO mapreduce.JobSubmitter: number of splits:2
18/11/07 16:59:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1541602145497_0030
18/11/07 16:59:55 INFO impl.YarnClientImpl: Submitted application application_1541602145497_0030
18/11/07 16:59:55 INFO mapreduce.Job: The url to track the job: http://5c62422b412d:8088/proxy/application_1541602145497_0030/
18/11/07 16:59:55 INFO mapreduce.Job: Running job: job_1541602145497_0030
18/11/07 17:00:01 INFO mapreduce.Job: Job job_1541602145497_0030 running in uber mode : false
18/11/07 17:00:01 INFO mapreduce.Job:  map 0% reduce 0%
18/11/07 17:00:17 INFO mapreduce.Job:  map 10% reduce 0%
18/11/07 17:00:23 INFO mapreduce.Job:  map 14% reduce 0%
18/11/07 17:00:29 INFO 